# Assignment 4: Isolation Forests
Kellen Mossner 26024284

#### Investigate parameter correlations & practical guidelines
Likely correlations to test

- max_samples vs n_estimators: smaller max_samples may need more n_estimators to stabilize detection.

- max_features vs dataset dimensionality: high-dim noisy features → lower max_features helps.

- contamination vs observed precision/recall: higher contamination → more positive predictions → recall ↑, precision ↓.

Guidelines to derive

- If anomalies are sparse and subtle → use larger max_samples (to let trees see enough structure) and moderate n_estimators.

- If anomalies are strong single-point outliers → smaller max_samples may isolate them faster (but validate for variance).

- For noisy high-dim data → lower max_features or feature selection prior to IF.

- Use n_estimators large enough to stabilize scores (check std of metric), but stop when marginal gain is tiny.

- If contamination is unknown, evaluate on score metrics (AUC/AP) and avoid hard thresholding without validation.

In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo 
from scipy.io import loadmat

from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold 
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    accuracy_score,
    classification_report,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
    matthews_corrcoef
)
import tqdm

# IEEE dual-column specifications
column_width_pt = 241.14734  # Exact IEEE dual-column width
text_width_pt = 516.0        # Full text width (both columns + gap)

column_width_inches = column_width_pt / 72.27
text_width_inches = text_width_pt / 72.27

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.size": 10,           # Base font size to match document
    "axes.labelsize": 10,      # x and y axis labels
    "axes.titlesize": 10,      # Plot title
    "xtick.labelsize": 10,      # x-axis tick labels (slightly smaller)
    "ytick.labelsize": 10,      # y-axis tick labels (slightly smaller)
    "legend.fontsize": 10,      # Legend text
    "figure.titlesize": 10,    # Figure title (if used)
    
    # Line and marker properties
    "lines.linewidth": 1.2,
    "lines.markersize": 4,
    "patch.linewidth": 0.5,
    
    # Grid properties
    "grid.linewidth": 0.5,
    "grid.alpha": 0.3,
    
    # Save properties
    "savefig.dpi": 300,
    "savefig.bbox": "tight",
    "savefig.format": "pdf",
    "savefig.pad_inches": 0.02,
})

import scikit_posthocs as sp
from scipy.stats import rankdata, wilcoxon
from scipy.stats import friedmanchisquare
from scipy.stats import rankdata
from scipy.stats import ttest_rel

In [66]:
# Data quality reports
def quality_report(df):
    """
    Generate a quality report for the DataFrame.
    """

    report_cts = df.describe(include=[np.number]).T
    report_cts["Cardinality"] = df.nunique()
    report_cts["Missing Percentage"] = (df.isnull().sum() / len(df)) * 100
    report_cts.rename(
        columns={
            "50%": "Median",
            "25%": "1st Qrt",
            "75%": "3rd Qrt",
            "mean": "Mean",
            "count": "Count",
            "max": "Max",
            "std": "Std Dev",
            "min": "Min",
        },
        inplace=True,
    )
    report_cts["Count"] = len(df)
    report_cts.reset_index(inplace=True)
    report_cts.rename(columns={"index": "Feature"}, inplace=True)

    categorical_cols = df.select_dtypes(include=["object"]).columns
    cat_data = []
    for col in categorical_cols:
        value_counts = df[col].value_counts()
        total_count = len(df[col])
        mode = value_counts.index[0] if len(value_counts) > 0 else None
        mode_freq = value_counts.iloc[0] if len(value_counts) > 0 else 0
        mode_pct = (mode_freq / total_count * 100) if total_count > 0 else 0
        second_mode = value_counts.index[1] if len(value_counts) > 1 else None
        second_mode_freq = value_counts.iloc[1] if len(value_counts) > 1 else 0
        second_mode_pct = (
            (second_mode_freq / total_count * 100) if total_count > 0 else 0
        )
        cat_data.append(
            {
                "Feature": col,
                "Count": total_count,
                "Missing Percentage": round(
                    (df[col].isnull().sum() / len(df)) * 100, 2
                ),
                "Unique": df[col].nunique(),
                "Mode": mode,
                "Mode Freq": mode_freq,
                "Mode %": round(mode_pct, 2),
                "2nd Mode": second_mode,
                "2nd Mode Freq": second_mode_freq,
                "2nd Mode %": round(second_mode_pct, 2),
                "Cardinality": df[col].nunique(),
            }
        )
    report_cat = pd.DataFrame(cat_data)

    # return both data quality reports
    return report_cts, report_cat

### Data Pre-processing

#### Steel Faults Dataset

In [68]:
# Steel Plates Faults dataset
steel_plates_faults = fetch_ucirepo(id=198) 

X_fault = steel_plates_faults.data.features
y_fault = steel_plates_faults.data.targets # one hot encoded labels

# convert one-hot encoded labels to single column
y_fault = y_fault.idxmax(axis=1)
print(y_fault.value_counts())
y_fault = y_fault.apply(lambda x: 1 if x == 'Other_Faults' else 0)
y_fault.name = 'Outlier_label'

df_fault = pd.concat([X_fault, y_fault], axis=1)
df_fault['Outlier_label'] = df_fault['Outlier_label'].astype(object)

Other_Faults    673
Bumps           402
K_Scratch       391
Z_Scratch       190
Pastry          158
Stains           72
Dirtiness        55
Name: count, dtype: int64


In [69]:
report_cts1, report_cat1 = quality_report(df_fault)

report_cts1.head(50)

,Feature,Count,Mean,Std Dev,Min,1st Qrt,Median,3rd Qrt,Max,Cardinality,Missing Percentage
0,X_Minimum,1941,5.711360e+02,5.206907e+02,0.0000,51.0000,4.350000e+02,1.053000e+03,1.705000e+03,962,0.0
1,X_Maximum,1941,6.179645e+02,4.976274e+02,4.0000,192.0000,4.670000e+02,1.072000e+03,1.713000e+03,994,0.0
2,Y_Minimum,1941,1.650685e+06,1.774578e+06,6712.0000,471253.0000,1.204128e+06,2.183073e+06,1.298766e+07,1939,0.0
3,Y_Maximum,1941,1.650739e+06,1.774590e+06,6724.0000,471281.0000,1.204136e+06,2.183084e+06,1.298769e+07,1940,0.0
4,Pixels_Areas,1941,1.893878e+03,5.168460e+03,2.0000,84.0000,1.740000e+02,8.220000e+02,1.526550e+05,920,0.0
5,X_Perimeter,1941,1.118552e+02,3.012092e+02,2.0000,15.0000,2.600000e+01,8.400000e+01,1.044900e+04,399,0.0
6,Y_Perimeter,1941,8.296600e+01,4.264829e+02,1.0000,13.0000,2.500000e+01,8.300000e+01,1.815200e+04,317,0.0
7,Sum_of_Luminosity,1941,2.063121e+05,5.122936e+05,250.0000,9522.0000,1.920200e+04,8.301100e+04,1.159141e+07,1909,0.0
8,Maximum_of_Luminosity,1941,1.301937e+02,1.869099e+01,37.0000,124.0000,1.270000e+02,1.400000e+02,2.530000e+02,100,0.0
9,Length_of_Conveyer,1941,1.459160e+03,1.445778e+02,1227.0000,1358.0000,1.364000e+03,1.650000e+03,1.794000e+03,84,0.0


In [70]:
report_cat1.head(10)

,Feature,Count,Missing Percentage,Unique,Mode,Mode Freq,Mode %,2nd Mode,2nd Mode Freq,2nd Mode %,Cardinality
0,Outlier_label,1941,0.0,2,0,1268,65.33,1,673,34.67,2


#### Ann Thyroid Dataset

In [71]:
df_thyroid = pd.read_csv('../data/annthyroid/annthyroid_unsupervised_anomaly_detection.csv', sep=';')

# drop last two columns
df_thyroid.drop(columns=df_thyroid.columns[-2:], inplace=True)

# convert outlier target to binary
df_thyroid['Outlier_label'] = df_thyroid['Outlier_label'].apply(lambda x: 1 if x == 'o' else 0)
df_thyroid['Outlier_label'] = df_thyroid['Outlier_label'].astype(object)

df_thyroid.head()

,Age,Sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,goitre,tumor,hypopituitary,psych,TSH,T3_measured,TT4_measured,T4U_measured,FTI_measured,Outlier_label
0,0.45,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,61.0,6.0,23.0,87.0,26.0,1
1,0.61,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,29.0,15.0,61.0,96.0,64.0,1
2,0.16,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,29.0,19.0,58.0,103.0,56.0,1
3,0.85,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,114.0,3.0,24.0,61.0,39.0,1
4,0.75,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,49.0,3.0,5.0,116.0,4.0,1


In [72]:
report_cts2, report_cat2 = quality_report(df_thyroid)

report_cts2.head(50)

,Feature,Count,Mean,Std Dev,Min,1st Qrt,Median,3rd Qrt,Max,Cardinality,Missing Percentage
0,Age,6916,0.595098,6.189326,0.0100,0.37000,0.5400,0.6700,515.0,98,0.0
1,Sex,6916,0.307548,0.461512,0.0000,0.00000,0.0000,1.0000,1.0,2,0.0
2,on_thyroxine,6916,0.134615,0.341337,0.0000,0.00000,0.0000,0.0000,1.0,2,0.0
3,query_on_thyroxine,6916,0.015616,0.123993,0.0000,0.00000,0.0000,0.0000,1.0,2,0.0
4,on_antithyroid_medication,6916,0.013158,0.113959,0.0000,0.00000,0.0000,0.0000,1.0,2,0.0
5,sick,6916,0.038317,0.191974,0.0000,0.00000,0.0000,0.0000,1.0,2,0.0
6,pregnant,6916,0.011278,0.105606,0.0000,0.00000,0.0000,0.0000,1.0,2,0.0
7,thyroid_surgery,6916,0.014315,0.118793,0.0000,0.00000,0.0000,0.0000,1.0,2,0.0
8,I131_treatment,6916,0.016773,0.128428,0.0000,0.00000,0.0000,0.0000,1.0,2,0.0
9,query_hypothyroid,6916,0.062753,0.242536,0.0000,0.00000,0.0000,0.0000,1.0,2,0.0


In [73]:
report_cat2.head(50)

,Feature,Count,Missing Percentage,Unique,Mode,Mode Freq,Mode %,2nd Mode,2nd Mode Freq,2nd Mode %,Cardinality
0,Outlier_label,6916,0.0,2,0,6666,96.39,1,250,3.61,2


#### Sat-image 2

In [74]:
satimage = loadmat('../data/satimage-2/satimage-2.mat')

# convert to dataframe
X_sat = pd.DataFrame(satimage['X'])
y_sat = pd.DataFrame(satimage['y'], columns=['Outlier_label'])
y_sat['Outlier_label'] = y_sat['Outlier_label'].astype(object)

df_sat = pd.concat([X_sat, y_sat], axis=1)

In [75]:
report_cts3, report_cat3 = quality_report(df_sat)
report_cts3.head(50)

,Feature,Count,Mean,Std Dev,Min,1st Qrt,Median,3rd Qrt,Max,Cardinality,Missing Percentage
0,0,5803,71.519387,12.232358,40.0,63.0,70.0,82.0,104.0,50,0.0
1,1,5803,88.111839,18.426402,27.0,75.0,89.0,103.0,137.0,83,0.0
2,2,5803,97.797174,16.235822,53.0,84.0,100.0,112.0,139.0,75,0.0
3,3,5803,78.912804,14.337176,33.0,68.0,80.0,90.0,146.0,91,0.0
4,4,5803,71.304325,12.173531,40.0,63.0,70.0,82.0,104.0,50,0.0
5,5,5803,87.811821,18.390712,27.0,75.0,88.0,103.0,137.0,81,0.0
6,6,5803,97.572635,16.246730,50.0,84.0,98.0,111.0,139.0,75,0.0
7,7,5803,78.742719,14.338564,29.0,67.0,79.0,89.0,157.0,91,0.0
8,8,5803,71.021368,12.090072,40.0,63.0,68.0,80.0,104.0,50,0.0
9,9,5803,87.354472,18.452842,27.0,75.0,88.0,103.0,130.0,79,0.0


In [76]:
report_cat3.head(10)

,Feature,Count,Missing Percentage,Unique,Mode,Mode Freq,Mode %,2nd Mode,2nd Mode Freq,2nd Mode %,Cardinality
0,Outlier_label,5803,0.0,2,0.0,5732,98.78,1.0,71,1.22,2
